# Feature - Interoperability with Official NumPy

In [ ]:
from mxnet import np, npx
npx.set_np()
import numpy as onp

shape = (3, 4)
a = np.random.uniform(size=shape)  # type of a is mxnet.numpy.ndarray
b = np.random.uniform(size=shape)  # type of b is mxnet.numpy.ndarray
print("type of a is", type(a))
print("type of b is", type(b))
c = onp.add(a, b)  # type of c is mxnet.numpy.ndarray
print("type of c is", type(c))

c_sum = onp.sum(c, axis=0)  # type of c_sum is mxnet.numpy.ndarray
print("type of c_sum is", type(c_sum))

# Feature - Parallel Acceleration

In [ ]:
import numpy as onp

lhs_shape = (4096, 1024)
rhs_shape = (1024, 4096)

onp_lhs = onp.random.uniform(-1.0, 1.0, size=lhs_shape).astype(np.float32)
onp_rhs = onp.random.uniform(-1.0, 1.0, size=rhs_shape).astype(np.float32)

onp_out = onp.dot(onp_lhs, onp_rhs)

## Running with official NumPy

In [ ]:
import numpy as onp

lhs_shape = (4096, 1024)
rhs_shape = (1024, 4096)

onp_lhs = onp.random.uniform(-1.0, 1.0, size=lhs_shape).astype(np.float32)
onp_rhs = onp.random.uniform(-1.0, 1.0, size=rhs_shape).astype(np.float32)

import time
onp_start = time.time()
onp_out = onp.dot(onp_lhs, onp_rhs)
onp_end = time.time()
print("official numpy consumed:", onp_end - onp_start, "seconds.")

## Running with NP on MXNet on CPU

In [ ]:
from mxnet import np, npx, nd
npx.set_np()

lhs_shape = (4096, 1024)
rhs_shape = (1024, 4096)


dnp_cpu_lhs = np.random.uniform(-1.0, 1.0, size=lhs_shape).astype(np.float32)
dnp_cpu_rhs = np.random.uniform(-1.0, 1.0, size=rhs_shape).astype(np.float32)

import time
dnp_cpu_start = time.time()
dnp_cpu_out = np.dot(dnp_cpu_lhs, dnp_cpu_rhs)
nd.waitall()
dnp_cpu_end = time.time()
print("NP on MXNet consumed:", dnp_cpu_end - dnp_cpu_start, "seconds on CPU.")

## Running with NP on MXNet on GPU

In [ ]:
from mxnet import np, npx, nd
npx.set_np()

lhs_shape = (4096, 1024)
rhs_shape = (1024, 4096)

if npx.num_gpus() >= 1:
    dnp_gpu_lhs = np.random.uniform(-1.0, 1.0, size=lhs_shape).as_in_ctx(npx.gpu(0))
    dnp_gpu_rhs = np.random.uniform(-1.0, 1.0, size=rhs_shape).as_in_ctx(npx.gpu(0))

    import time
    dnp_gpu_start = time.time()
    dnp_gpu_out = np.dot(dnp_gpu_lhs, dnp_gpu_rhs)
    nd.waitall()
    dnp_gpu_end = time.time()
    print("NP on MXNet consumed:", dnp_gpu_end - dnp_gpu_start, "seconds on GPU.")

## Activation Functions

In [ ]:
from mxnet import np, npx
npx.set_np()

x = np.linspace(-4., 4.)
sigmoid = npx.sigmoid(x)
relu = npx.relu(x)

%matplotlib inline
import matplotlib.pyplot as plt

axes = plt.gca()
axes.set_ylim([-0.1,1.0])
plt.xlabel("x"); plt.ylabel("f(x)")
plt.plot(x, sigmoid, label='Sigmoid')
plt.plot(x, relu, label='ReLU')
axes.legend(fontsize='xx-large')

## Batch Normalization Function
![BN_backward](https://kratzert.github.io/images/bn_backpass/BNcircuit.png)
*figure credit: [Understanding the backward pass through Batch Normalization Layer](https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html)

In [ ]:
import numpy as onp
def batchnorm_forward(x, gamma, beta, eps=1e-5):
    N = x.shape[0]
    #step1: calculate mean
    mu = onp.mean(x, axis = 0)
    #step2: subtract mean vector of every trainings example
    xmu = x - mu
    #step3: following the lower branch - calculation denominator
    sq = xmu ** 2
    #step4: calculate variance
    var = onp.mean(sq, axis = 0)
    #step5: add eps for numerical stability, then sqrt
    sqrtvar = onp.sqrt(var + eps)
    #step6: invert sqrtwar
    ivar = 1./sqrtvar
    #step7: execute normalization
    xhat = xmu * ivar
    #step8: Nor the two transformation steps
    gammax = gamma * xhat
    #step9
    out = gammax + beta
    #store intermediate
    cache = (xhat, gamma, xmu, ivar, sqrtvar, var, eps)
    return out, cache

## Compute with NP on MXNet

In [ ]:
from mxnet import np, npx
npx.set_np()

shape = (32, 784)

batch_size = shape[0]

data = np.random.uniform(-1.0, 1.0, size=shape)
gamma = np.random.uniform(-1.0, 1.0, size=shape[1:])
beta = np.random.uniform(-1.0, 1.0, size=shape[1:])
dnp_out, dnp_cache = batchnorm_forward(data, gamma, beta)

## Computing with Official NumPy

In [ ]:
onp_data = data.asnumpy()
onp_gamma = gamma.asnumpy()
onp_beta = beta.asnumpy()
onp_out, onp_cache = batchnorm_forward(onp_data, onp_gamma, onp_beta)

## Checking Consistency

In [ ]:
from mxnet.test_utils import assert_almost_equal
# assert_almost_equal is a mxnet helper function
# that is similar to numpy.testing.assert_almost_equal,
# it will raise error if the 2 inputs do not match
try:
    assert_almost_equal(dnp_out, onp_out, atol=1e-5, rtol=1e-3)
    for dnp_arr, onp_arr in zip(dnp_cache, onp_cache):
        assert_almost_equal(dnp_arr, onp_arr, atol=1e-5, rtol=1e-3)
    print("the outputs match!")
except:
    print("the outputs do not match!")

## Backward Propagation
![BN_backward](https://kratzert.github.io/images/bn_backpass/BNcircuit.png)
*figure credit: [Understanding the backward pass through Batch Normalization Layer](https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html)

## Computing Gradient with Official NumPy 

In [ ]:
import numpy as onp
def batchnorm_backward(dout, cache):
    # requires the intermediate values for faster computations
    xhat, gamma, xmu, ivar, sqrtvar, var, eps = cache
    N = dout.shape[0]  # get the batch size

    dbeta = onp.sum(dout, axis=0)  # step 9
    dgammax = dout  # not necessary, but more understandable

    dgamma = onp.sum(dgammax*xhat, axis=0)  #step 8
    dxhat = dgammax * gamma  # step 8

    divar = onp.sum(dxhat*xmu, axis=0)  # step 7
    dxmu1 = dxhat * ivar  # step 7

    dsqrtvar = -divar / (sqrtvar**2)  # step 6

    dvar = 0.5 / onp.sqrt(var + eps) * dsqrtvar  # step5
    
    dsq = onp.ones(dout.shape, dtype=onp.float32) * dvar / N  # step 4
    
    dxmu2 = 2 * xmu * dsq  # step 3

    dx1 = dxmu1 + dxmu2  # step 2
    dmu = -onp.sum(dx1, axis=0)  # step 2

    dx2 = onp.ones(dout.shape, dtype=onp.float32) * dmu / N  # step 1

    dx = dx1 + dx2  # step 0

    return dx, dgamma, dbeta

In [ ]:
from mxnet import np, npx
dout = np.random.uniform(size=onp_out.shape)

onp_dout = dout.asnumpy()
# forward pass
onp_out, onp_cache = batchnorm_forward(onp_data, onp_gamma, onp_beta)
# backward pass
onp_dx, onp_dgamma, onp_dbeta = batchnorm_backward(onp_dout, onp_cache)

## Computing Gradient with Automatic Differentiation

In [ ]:
from mxnet import autograd

# attach gradient buffers to the arrays we want to compute
# gradient for
data.attach_grad()
gamma.attach_grad()
beta.attach_grad()

with autograd.record():  # autograd.record() gives a scope which captures
                         # code that needs gradient computation
    dnp_out, dnp_cache = batchnorm_forward(data, gamma, beta)

dnp_out.backward(out_grad=dout)  # compute the gradients with respect to dnp_out's
                                 # variables. In this case, dnp_out is computed by
                                 # batchnorm_forward(data, gamma, beta)
                                 # so data, gamma and beta are out's variables

# data's gradient now in data.grad
# gamma's gradient now in gamma.grad
# beta's gradient now in beta.grad

## Checking Consistency

In [ ]:
from mxnet.test_utils import assert_almost_equal
# assert_almost_equal is a mxnet helper function
# that is similar to numpy.testing.assert_almost_equal,
# it will raise error if the 2 inputs do not match

try:
    assert_almost_equal(data.grad, onp_dx, atol=1e-5, rtol=1e-3)
    assert_almost_equal(gamma.grad, onp_dgamma, atol=1e-5, rtol=1e-3)
    assert_almost_equal(beta.grad, onp_dbeta, atol=1e-5, rtol=1e-3)
    print("the gradients match!")
except:
    print("the gradients do not match!")

## Building a model with NP on MXNet

## Getting the Environment Ready

In [ ]:
# import np and npx modules
from mxnet import np, npx
# setting MXNet to NumPy-compatible mode
npx.set_np()
# we'll be using automatic differentiation
from mxnet import autograd
# borrowing the dataloader from our Gluon library
from mxnet import gluon

## Prepare the Dataloader

In [ ]:
batch_size = 256  # setting the batch size
def load_data_fashion_mnist(batch_size):
    dataset = gluon.data.vision
    trans = [dataset.transforms.ToTensor()]
    trans = dataset.transforms.Compose(trans)
    mnist_train = dataset.FashionMNIST(train=True).transform_first(trans)
    mnist_test = dataset.FashionMNIST(train=False).transform_first(trans)
    return (gluon.data.DataLoader(mnist_train, batch_size, shuffle=True,
                                  num_workers=4),
            gluon.data.DataLoader(mnist_test, batch_size, shuffle=False,
                                  num_workers=4))
train_iter, test_iter = load_data_fashion_mnist(batch_size)
# train_iter and test_iter are dataset iterators
# for training set and test set respectively

## Initializing all Model Parameters

In [ ]:
# a 2-layer multi-layer perceptron with batch normalization
num_inputs, num_outputs, num_hiddens = 784, 10, 256

# layer 1 weight
W1 = np.random.normal(scale=0.01, size=(num_inputs, num_hiddens))
# layer 1 bias
b1 = np.zeros(num_hiddens)
# gamma parameter for batch normalization
gamma = np.ones(shape=(num_hiddens,))
# beta parameter for batch normalization
beta = np.zeros(shape=(num_hiddens,))
# layer 2 weight
W2 = np.random.normal(scale=0.01, size=(num_hiddens, num_outputs))
# layer 2 bias
b2 = np.zeros(num_outputs)
# collect all parameter for easier access
params = [W1, b1, gamma, beta, W2, b2]

for param in params:
    param.attach_grad()  # attach gradient to all parameters
                         # for using automatic differentiation

## Defining the model

In [ ]:
def relu(X):
    # ReLU activation function: f(x) = x if x > 0 else 0
    return np.maximum(X, 0)

def batchnorm(x, gamma, beta, eps=1e-5):
    N = x.shape[0]

    mu = np.mean(x, axis = 0)
    xmu = x - mu

    sq = xmu ** 2
    var = np.mean(sq, axis = 0)
    sqrtvar = np.sqrt(var + eps)
    ivar = 1. / sqrtvar

    xhat = xmu * ivar

    gammax = gamma * xhat
    out = gammax + beta

    return out

def net(X):
    X = X.reshape(-1, num_inputs)  # (batch_size, 28, 28) -> (batch_size, 784)
    H = relu(np.dot(X, W1) + b1)   # H: (batch_size, 256)
    H_normed = batchnorm(H, gamma, beta) # H_normed: (batch_size, 256)
    return np.dot(H_normed, W2) + b2     # output: (batch_size, 10)

## Setting up Loss and Optimizer Function

In [ ]:
def softmax(y_hat):
    # stable version of softmax function
    exps = np.exp(y_hat - np.max(y_hat, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

def loss(y_hat, y):
    m = y.shape[0]
    p = softmax(y_hat)
    # cross entropy = -sum(y_i * log(p(y_i)))
    return np.sum(-np.log(p[range(m), y]))

# stochastic gradient descent
def sgd(params, lr, wd, batch_size):
    for param in params:
        param[:] = param - lr * param.grad / batch_size - param * wd

## Model Evaluation Metrics

In [ ]:
def accuracy(y_hat, y):
    # y_hat: (batch_size, 10)
    # y:     (batch_size, 1)
    # label prediction of the model is the index
    # of the maximum value for each sample
    return float((y_hat.argmax(axis=1) == y.astype('float32')).sum())

def evaluate_accuracy(net, data_iter):
    num_correct_example = 0
    total_num_example = 0

    for X, y in data_iter:
        num_correct_example += accuracy(net(X), y)
        total_num_example += y.size
    # return the percentage of correct predictions
    return num_correct_example / total_num_example

## Training the Model

In [ ]:
num_epochs = 10  # training for 10 epochs
lr = 0.5         # with learning rate of 0.5
wd = 0.001       # with weight decay of 0.001

for epoch in range(num_epochs):  # for each epoch
    train_loss = 0.0
    train_acc = 0
    num_examples = 0
    for X, y in train_iter:
        with autograd.record():
            y_hat = net(X)       # forward pass
            l = loss(y_hat, y)   # computing loss
        l.backward()
        # at this moment the gradients are ready in the
        # gradient buffers of all parameters in params
        sgd(params, lr, wd, X.shape[0])  # update parameters
        train_loss += l.sum()            # accumulate loss
        train_acc += accuracy(y_hat, y)  # accumulate accuracy
        num_examples += y.size           # count total number of samples
    # Return training loss and training accuracy
    train_loss, train_acc = train_loss/num_examples, train_acc/num_examples
    test_acc = evaluate_accuracy(net, test_iter)
    print("epoch {}: train loss {} "
          "train accuracy {} test accuracy {}".format(epoch,
                                                      train_loss,
                                                      train_acc,
                                                      test_acc))